In [ ]:
import serial
import time
import csv
# COM3 are the actual port of my ardino
# Replace 'COM3' with your actual port (e.g., 'COM3' on Windows or '/dev/ttyUSB0' on Linux)
# Make sure to install pyserial: pip install pyserial
ser = serial.Serial('COM3', 9600, timeout=2)
time.sleep(2)  # Wait for Arduino to reset

def send_command(cmd):
    print(f">> Sending: {cmd}")
    ser.write((cmd + '\n').encode())
    while True:
        response = ser.readline().decode().strip()
        if response:
            print("<< Arduino:", response)
            break
# Define movement commands
def move_to(x, y, z):
    send_command(f"MOVE {x} {y} {z}")
# Define home position command
def go_home():
    send_command("HOME")
# Define gripper commands
def gripper_close():
    send_command("GRIPPER 0")
def gripper_open():
    send_command("GRIPPER 1")
# Function to compensate for camera distortion based on coordinates
# This function assumes the camera is mounted above the foam block and compensates for distortion
def camera_compensation(x_coordinate, y_coordinate):
    h_foam = 60  # foam height in mm
    camera_position = [390, 45, 660]  # [x, y, z] position of the camera in mm
    offset = 390  # shift origin to be under the camera

    # Step 1: Recenter x based on camera position and offset
    x_coordinate = (offset - x_coordinate) + (camera_position[0] - offset)

    # Step 2: Compensate for distortion caused by height (x-axis)
    x_compensated = x_coordinate - (h_foam / (camera_position[2] / x_coordinate))

    # Step 3: Compensate for distortion caused by height (y-axis)
    if y_coordinate < camera_position[1]:
        y_compensated = y_coordinate - (h_foam / (camera_position[2] / y_coordinate))
    else:
        y_compensated = y_coordinate + (h_foam / (camera_position[2] / y_coordinate))

    # Step 4: Transform x back to original orientation
    x_compensated = offset - (x_compensated - (camera_position[0] - offset))

    return int(x_compensated-390), int(y_compensated)

def save_coordinates_to_csv(cup_x, cup_y, target_x, target_y):
    """Save coordinates to CSV file with test index and timestamp"""
    global test_index
    csv_filename = 'coordinates_log.csv'
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([test_index, cup_x, cup_y, target_x, target_y])
    
    print(f"Test #{test_index} coordinates saved to {csv_filename}")
    print(f"Cup: ({cup_x}, {cup_y}), Target: ({target_x}, {target_y})")
    
    test_index += 1  # Increment for next test
print("Connection closed.")
ser.close()

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import serial
import time
import csv
import os
from datetime import datetime

ser = serial.Serial('COM3', 9600, timeout=2)
time.sleep(2)  # Wait for Arduino to reset

# Load YOLO model
model = YOLO("my_model.pt")

# Open video capture
cap = cv2.VideoCapture(1)

# ArUco dictionary and detector
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

# Real-world target size (in mm)
WIDTH_MM = 500
HEIGHT_MM = 250

# Simple storage
saved_objects = []

# CSV file setup
csv_filename = 'coordinates_log.csv'
csv_headers = ['test_index', 'cup_x', 'cup_y', 'target_x', 'target_y']

# Initialize test counter
test_index = 1

# Create CSV file with headers if it doesn't exist, or get the next test index
if not os.path.exists(csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(csv_headers)
else:
    # Read existing file to get the next test index
    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            rows = list(reader)
            if len(rows) > 1:  # If there are data rows (not just headers)
                last_row = rows[-1]
                test_index = int(last_row[0]) + 1
    except (ValueError, IndexError):
        test_index = 1

def save_coordinates_to_csv(cup_x, cup_y, target_x, target_y):
    """Save coordinates to CSV file with test index and timestamp"""
    global test_index
        
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([test_index, cup_x, cup_y, target_x, target_y])
    
    print(f"Test #{test_index} coordinates saved to {csv_filename}")
    print(f"Cup: ({cup_x}, {cup_y}), Target: ({target_x}, {target_y})")
    
    test_index += 1  # Increment for next test

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_objects = {}  # Objects in current frame

    # --- ArUco detection ---
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = detector.detectMarkers(gray)

    if ids is not None and len(ids) >= 4:
        # Map ids to corners
        id_to_corner = {id[0]: corner[0] for id, corner in zip(ids, corners)}
        # Sort corners by ID
        selected_ids = sorted(id_to_corner.keys())[:4]
        # Extract corners for the selected markers
        selected_corners = [id_to_corner[i] for i in selected_ids]
        # Calculate the center of each marker
        marker_centers = [np.mean(pts, axis=0) for pts in selected_corners]

        marker_centers = np.array(marker_centers)
        s = marker_centers.sum(axis=1)
        diff = np.diff(marker_centers, axis=1)

        sorted_pts = np.zeros((4, 2), dtype="float32")
        sorted_pts[0] = marker_centers[np.argmin(s)]     # Top-left
        sorted_pts[2] = marker_centers[np.argmax(s)]     # Bottom-right
        sorted_pts[1] = marker_centers[np.argmin(diff)]  # Top-right
        sorted_pts[3] = marker_centers[np.argmax(diff)]  # Bottom-left

        dst_pts = np.array([
            [0, 0],
            [WIDTH_MM - 1, 0],
            [WIDTH_MM - 1, HEIGHT_MM - 1],
            [0, HEIGHT_MM - 1]
        ], dtype="float32")

        # Warp perspective
        matrix = cv2.getPerspectiveTransform(sorted_pts, dst_pts)
        warped = cv2.warpPerspective(frame, matrix, (WIDTH_MM, HEIGHT_MM))

        # --- Run YOLO on warped frame ---
        results = model(warped)[0]

        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            w, h = x2 - x1, y2 - y1
            class_name = model.names[int(class_id)]

            # Calculate the center of the bounding box
            center_x = x1 + w // 2
            center_y = y1 + h // 2

            # Store current object info
            current_objects[class_name] = {
                    'x': center_x,
                    'y': center_y,
                    'confidence': score
                }

            # Draw bounding box (1 pixel = 1 mm)
            cv2.rectangle(warped, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Mark the center with a circle
            cv2.circle(warped, (center_x, center_y), 5, (0, 0, 255), -1)  # Red dot for center

            label = f"{class_name} ({int(score * 100)}%)"
            coord_text = f"X:{center_x}mm Y:{center_y}mm W:{w}mm H:{h}mm"

            cv2.putText(warped, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(warped, coord_text, (x1, y2 + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Show instructions
        cv2.putText(warped, "Press 'a' to save coordinates", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

        # Show warped detection result
        cv2.imshow("Warped Detection (mm coordinates)", warped)

    # Handle key presses
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == ord('Q'):
        break
    elif key == ord('a') or key == ord('A'):
        # Save current objects
        if current_objects:
            try:
                cup_x = current_objects["cup"]['y']
                cup_y = current_objects["cup"]['x'] - 250 #offset the original coordinates to the robot same as the ardrino inverse kinematics
                target_x = current_objects["target"]['y']
                target_y = current_objects["target"]['x'] - 250 #offset the original coordinates to the robot same as the ardrino inverse kinematics
                # Save coordinates to CSV
                save_coordinates_to_csv(cup_x, cup_y, target_x, target_y)
                if target_y < 0:
                    target_y = target_y -5  #additional offset if target is on the left side
                elif target_y > 0:
                    target_y = target_y +5
                else:
                    target_y = target_y # no offset if target is in the middle
                #target_x, target_y = camera_compensation(target_x, target_y)  # Compensate for camera distortion
                gripper_open()  # Open gripper
                move_to(target_x+15,target_y, 20)  # Move to target position
                gripper_close()
                move_to(target_x+15,target_y, 120)  # Move on top of target position
                move_to(cup_x, cup_y, 110)#move on top of cup position
                gripper_open()
                go_home()  # Go back to home position
                gripper_close()
            except KeyError as e:
                print(f"Error: {e} not detected or invalid coordinates")
            except Exception as e:
                print(f"Unexpected error: {e}")
                
cap.release()
cv2.destroyAllWindows()
go_home()
ser.close()